In [1]:
import requests
import uuid
import json
import time

# ПОЛУЧАЕМ ДАННЫЕ О СПИСКЕ КОЛЛЕКЦИЙ

In [13]:
id = str(uuid.uuid4())
message = { "id": id, "mqtt_key": "234", "user": "2321", "type": "courses", "data_key": "","action": "get_data" }
resp = requests.post("http://localhost:8000/api/add_message", json=message)
resp

<Response [200]>

In [14]:
message = { "id": id }
resp = requests.post("http://localhost:57888/api/get_message_result", json=message)
resp.json()

{'message': {'id': '0f54f511-6152-46c6-ab1d-12b393b861c9',
  'mqtt_key': '234',
  'user': '2321',
  'type': 'courses',
  'data_key': '',
  'action': 'get_data',
  'timestamp': 1669655171},
 'result': {'courses': ['test']}}

# ПОЛУЧАЕМ ДАННЫЕ О СТРУКТУРЕ КОЛЛЕКЦИИ

In [27]:
id = str(uuid.uuid4())
message = { "id": id, "mqtt_key": "234", "user": "2321", "type": "problems", "data_key": "test", "action": "get_data" }
resp = requests.post("http://cluster.vstu.ru:57888/api/add_message", json=message)
resp

<Response [200]>

In [29]:
message = { "id": id }
resp = requests.post(
    "http://cluster.vstu.ru:57888/api/get_message_result", json=message)
resp


<Response [200]>

In [30]:
resp.json()

{'message': {'id': 'bb56af03-e83e-493e-9a2e-1a73bf80a1f4',
  'mqtt_key': '234',
  'user': '2321',
  'type': 'problems',
  'data_key': 'test',
  'action': 'get_data',
  'timestamp': 1669822065},
 'result': {'problems': [{'1': ['1', '2'],
    'task': 'Задача - написать алгоритм, который получает на вход целое число и возвращает число на 1 большее.'},
   {'2': ['1', '2'],
    'task': 'Задача - написать алгоритм, который получает на вход целое число и возвращает число на 1 большее.'}]}}

# ДОБАВЛЕНИЕ ЗАДАЧ ИЗ ФАЙЛА

In [33]:
course = "test"
addr = "http://cluster.vstu.ru:57888"
with open("test.json", encoding='utf-8') as f:
    data = json.load(f)
for p in data:
    for v in p["variants"]:
        print(f'Problem {p["problem"]}, variant {v}')
        id = str(uuid.uuid4())
        message = {
            "id": id, 
            "mqtt_key": 123,
            "user": "admin", 
            "type": p["type"], 
            "course": course, 
            "problem": p["problem"], 
            "variant": v,
            "action": "add_problem",
            "task": p.get("task", ""),
            "tests": p["variants"][v]
        }
        r = requests.post(f"{addr}/api/add_message", json=message)
        print(f"{r}")
        message = { "id": id }
        while True:
            time.sleep(1)
            resp = requests.post(f"{addr}/api/get_message_result", json=message)
            if resp.status_code == 200 and 'message' in resp.text:
                break
                print(f"{resp.json()}")


Problem 1, variant 1
<Response [200]>
Problem 1, variant 2
<Response [200]>
Problem 2, variant 1
<Response [200]>
Problem 2, variant 2
<Response [200]>
